In [ ]:
import torch 
import torchvision 
import torchvision.transforms as transforms

from PIL import Image, ImageFilter
import cv2
import numpy as np


In [ ]:

tensor_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=tensor_transform)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)



In [ ]:

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = Net()



In [ ]:

net.load_state_dict(torch.load('fgsm_trained.pth'))
net.eval()

criterion = nn.CrossEntropyLoss()



In [ ]:

def FGSM_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon*sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def denorm(batch, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
    if isinstance(mean, list):
        mean = torch.tensor(mean)
    if isinstance(std, list):
        std = torch.tensor(std)
    return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)



In [ ]:


def FGSM( model, test_dataloader, epsilon ):
    total = 0
    correct = 0
    for data, target in test_dataloader:
        data.requires_grad = True
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]
        for i in range(len(target)):
            total+=1
            loss = criterion(output, target)
            model.zero_grad()
            loss.backward()
            data_grad = data.grad.data
            data_denorm = denorm(data)
            perturbed_data = FGSM_attack(data_denorm, epsilon, data_grad)
            perturbed_data_normalized = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(perturbed_data)

            preprocessed_images = nlmdenoising(perturbed_data_normalized) #change the function name to the required preprocessing method
            output = model(preprocessed_images)
            
            #output = model(perturbed_data_normalized)
            final_pred = output.max(1, keepdim=True)[1]
            if final_pred[i].item() == target[i].item():
                correct += 1

    print(f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {total} = {100 * correct / total} %")



In [ ]:
#Gaussian Blur

def gaussianblur(images):
    blurred_images_batch = []
    for image in images:
        transform1 = transforms.ToPILImage()
        pil_image = transform1(image)
        blurred_image = pil_image.filter(ImageFilter.GaussianBlur(radius=0.1))
        transform2 = transforms.ToTensor()
        tensor_blurred_image = transform2(blurred_image)
        blurred_images_batch.append(tensor_blurred_image)
    blurred_images_batch = torch.stack(blurred_images_batch)
    return blurred_images_batch



In [ ]:
#Median filtering

def medianfilter(images):
    median_images_batch = []
    for image in images:
        transform1 = transforms.ToPILImage()
        pil_image = transform1(image)
        median_image = pil_image.filter(ImageFilter.MedianFilter(size = 1))
        transform2 = transforms.ToTensor()
        tensor_median_image = transform2(median_image)
        median_images_batch.append(tensor_median_image)
    median_images_batch = torch.stack(median_images_batch)
    return median_images_batch



In [ ]:
#Non Local Means Denoising

def nlmdenoising(images):
    nlm_images_batch = []
    for image in images:
        transform1 = transforms.ToPILImage()
        pil_image = transform1(image)

        h=10
        np_image = np.array(pil_image)
        denoised_image = cv2.fastNlMeansDenoisingColored(np_image, None, h , h , templateWindowSize = 7, searchWindowSize = 21)
        nlm_image = Image.fromarray(denoised_image)
        
        transform2 = transforms.ToTensor()
        tensor_nlm_image = transform2(nlm_image)
        nlm_images_batch.append(tensor_nlm_image)
    nlm_images_batch = torch.stack(nlm_images_batch)
    return nlm_images_batch

In [ ]:

epsilons = [0, .05, .1, .15, .2, .25, .3]

for e in epsilons:
    FGSM(net, test_dataloader, e)

